In [ ]:
##Python venv: search_agent_poc

In [2]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like identifying 
## 1.Important Objects/Entities
## 2.Deduplicate Entities
## 3.Extracting Relations
## 4.Extract the main Ideas/Topics around Each Page
## 5.Link the different topics via diffrent entities/Objects
## 6.Break down the document by pages instead of Chunks .
## 7.If a page does not fit a chunk then chunk them extract information and then deduplicate the information across
## the page.

#Next Steps:
## 5.Try to Seggregate the BigPDF on Sections.
## 8.Try To Find Common Objects or ideas that link these sections.
## 9.Try 

In [10]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pickle
import ollama
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

import pandas as pd
#from langchain_community.vectorstores import Chroma

In [11]:

single_entity_file="nodes_with_single_ocurence.csv"

client = chromadb.PersistentClient(
    path="./Kubernetes_in_Action_ChromaDB",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)
#collection = client.delete_collection(name="entities_db_1")
#collection1 = client.create_collection(name="entities_db_2")
#collection_desc = client.create_collection(name="entities_db_desc")

In [12]:
#entity_idx=0
def embedder(index,text,collection):
    print(text)
    response = ollama.embeddings(model="mxbai-embed-large", prompt=text)
    embedding = response["embedding"]
    #print(np.array(embedding))
    try:
        collection.add(
            ids=[str(index)],
            embeddings=[embedding],
            documents=[text]
          )
    except:
          'COULD NOT LOAD INTO VECTOR DATABASE'
    #entity_idx=entity_idx+1
    return embedding

In [13]:
graph_output_dir='./graph_output_networkx/'
single_entity_file='nodes_with_single_ocurence.csv'
single_df=pd.read_csv(graph_output_dir+single_entity_file)
single_df=single_df.reset_index(drop=True).reset_index().dropna()
single_df['index']

0          0
1          1
2          2
3          3
4          4
        ... 
1843    1843
1844    1844
1845    1845
1846    1846
1847    1847
Name: index, Length: 1826, dtype: int64

In [14]:
single_df_final=single_df[['index','entity','description','category','node_occurence_flag']].dropna()
single_df_final

,index,entity,description,category,node_occurence_flag
0,0,UTS namespace,Shared hostname and network interfaces among c...,network,Single
1,1,Port space,Shared port numbers among containers in a pod,network,Single
2,2,Flat inter-pod network,"single flat, shared network-address space for ...",network,Single
3,3,NAT (Network Address Translation),gateway between pods that translates IP addresses,software,Single
4,4,Worker nodes,physical or virtual machines that run pods,hardware,Single
...,...,...,...,...,...
1843,1843,current-context,name of the current kubectl context being used,process,Single
1844,1844,certificate-authority,field in the cluster definition that specifies...,database,Single
1845,1845,/home/luksa/.minikube/ca.crt,path to the certificate authority file,hardware,Single
1846,1846,https://192.168.99.100:8443,address of the Kubernetes server,network,Single


In [15]:
collection_entity = client.delete_collection(name="entities_db_entity")
collection_entity = client.create_collection(name="entities_db_entity")

In [16]:
single_df_final.head(2)

,index,entity,description,category,node_occurence_flag
0,0,UTS namespace,Shared hostname and network interfaces among c...,network,Single
1,1,Port space,Shared port numbers among containers in a pod,network,Single


In [17]:
def chroma_collection_maker(collection_name,docs_lst):
    idx=0
    for item in docs_lst:
        print('Item:')
        print(item)
        print('index:')
        print(idx)
        response = ollama.embeddings(model="mxbai-embed-large", prompt=item)
        embedding = response["embedding"]
        #print(np.array(embedding))
    
        collection_name.add(
            ids=[str(idx)],
            embeddings=[embedding],
            documents=[item]
            )
    
    
        idx=idx+1
        
    return collection_name

def query_collection(collection_name,query,n):
    query = query
    # generate an embedding for the prompt and retrieve the most relevant doc
    response = ollama.embeddings(
      prompt=query,
      model="mxbai-embed-large"
    )
    results = collection_name.query(
      query_embeddings=[response["embedding"]],
      n_results=n
    )
    return results

In [18]:
collection_entity = client.delete_collection(name="entities_entity")
collection_entity = client.get_or_create_collection(name="entities_entity")

In [19]:
docs_lst=list(set(list(single_df_final['entity'])))
collection_entity=chroma_collection_maker(collection_entity,docs_lst)

Item:
/etc/hosts
index:
0
Item:
files in a volume
index:
1
Item:
App
index:
2
Item:
publishNotReadyAddresses
index:
3
Item:
egress rules
index:
4
Item:
BACKEND_SERVICE_HOST
index:
5
Item:
ReplicaSet v1
index:
6
Item:
servicebinding
index:
7
Item:
fortune
index:
8
Item:
instanceRef
index:
9
Item:
period=10s
index:
10
Item:
selector.app
index:
11
Item:
Clojure
index:
12
Item:
readinessProbe
index:
13
Item:
Admission Control plugins
index:
14
Item:
Pod A-2
index:
15
Item:
PATCH /v2/service_instances/:id
index:
16
Item:
lastState
index:
17
Item:
spec.containers
index:
18
Item:
nginx -s reload
index:
19
Item:
touch
index:
20
Item:
Ambassador containers
index:
21
Item:
kubectl top node command
index:
22
Item:
TCP connection
index:
23
Item:
kubectl create command
index:
24
Item:
yaml manifest
index:
25
Item:
Node X
index:
26
Item:
Affinity or anti-affinity rules
index:
27
Item:
BuildConfigs
index:
28
Item:
Delete
index:
29
Item:
container name
index:
30
Item:
video
index:
31
Item:
centralized

Item:
Extended Resources
index:
256
Item:
$ sudo ifconfig eth0 down
index:
257
Item:
--force
index:
258
Item:
Container Runtime
index:
259
Item:
while
index:
260
Item:
prefix
index:
261
Item:
/podName
index:
262
Item:
req
index:
263
Item:
frontend-941083-wsjv8
index:
264
Item:
SIGINT
index:
265
Item:
backend-257820-ssrgj
index:
266
Item:
fileExists()
index:
267
Item:
hostPath volumes
index:
268
Item:
returnedData
index:
269
Item:
gcloud command
index:
270
Item:
Base64 encoding
index:
271
Item:
OpenShift Online Starter
index:
272
Item:
user management
index:
273
Item:
Pod metadata
index:
274
Item:
tutum/dnsutils container image
index:
275
Item:
KubernetesClient
index:
276
Item:
DeploymentConfigs
index:
277
Item:
Mem
index:
278
Item:
$(VAR) syntax
index:
279
Item:
memory usage
index:
280
Item:
HPA (Horizontal Pod Autoscaler)
index:
281
Item:
requests.memory 10Mi
index:
282
Item:
LimitRanger Admission Control plugin
index:
283
Item:
MustRunAsNonRoot
index:
284
Item:
fortune binary
index:


Item:
FIRST_VAR
index:
501
Item:
ranges
index:
502
Item:
RWX
index:
503
Item:
PostgreSQL
index:
504
Item:
Service Endpoints
index:
505
Item:
postgres-database
index:
506
Item:
labels and annotations
index:
507
Item:
StatefulSets scale down
index:
508
Item:
https://github.com/yanatan16/clj-kubernetes-api
index:
509
Item:
nameserver
index:
510
Item:
production values
index:
511
Item:
Unknown
index:
512
Item:
tty10
index:
513
Item:
metrics field
index:
514
Item:
chapter 16
index:
515
Item:
items
index:
516
Item:
kubernetes.io/hostname
index:
517
Item:
-k option
index:
518
Item:
manning
index:
519
Item:
service accounts
index:
520
Item:
authorization token
index:
521
Item:
Data processor
index:
522
Item:
--cascade=false
index:
523
Item:
sysadmin
index:
524
Item:
validation schema
index:
525
Item:
docker.io/luksa/fortune:args
index:
526
Item:
syncHandler
index:
527
Item:
get instance
index:
528
Item:
Metrics
index:
529
Item:
app=pc
index:
530
Item:
PersistentDisk (PD)
index:
531
Item:
statu

Item:
core
index:
751
Item:
Terminating
index:
752
Item:
Supplemental Group IDs
index:
753
Item:
pref
index:
754
Item:
logVol volume
index:
755
Item:
stream of modifications
index:
756
Item:
--leader-elect option
index:
757
Item:
Kubernetes services
index:
758
Item:
vsphere-Volume
index:
759
Item:
$ kubectl get po kubia-0 -o yaml
index:
760
Item:
Unknown status
index:
761
Item:
shopping cart
index:
762
Item:
veth1178d4f
index:
763
Item:
Version
index:
764
Item:
Autoscaler controller
index:
765
Item:
/healthz
index:
766
Item:
Queries-Per-Second (QPS)
index:
767
Item:
/api/v1/pods
index:
768
Item:
ui pod
index:
769
Item:
FOO=BAR
index:
770
Item:
-s option
index:
771
Item:
dnsPolicy
index:
772
Item:
devices
index:
773
Item:
Google Cloud Monitoring
index:
774
Item:
Kubernetes resources
index:
775
Item:
Java app
index:
776
Item:
/path/to/dir
index:
777
Item:
ConfigMap keys
index:
778
Item:
external-service
index:
779
Item:
GKE (Google Kubernetes Engine)
index:
780
Item:
8080
index:
781
Item

Item:
Kubernetes Service resources
index:
1003
Item:
RollingUpdate strategy
index:
1004
Item:
ordinal number
index:
1005
Item:
dstIP: 10.1.2.1
index:
1006
Item:
probe
index:
1007
Item:
/api/v1/namespaces/default/pods/kubia-0/proxy/<path>
index:
1008
Item:
LabelSelector
index:
1009
Item:
true
index:
1010
Item:
ServiceAccount token
index:
1011
Item:
Frontend process
index:
1012
Item:
FOO_SECRET
index:
1013
Item:
/apis/batch
index:
1014
Item:
scheduling jobs
index:
1015
Item:
set selector
index:
1016
Item:
database pod
index:
1017
Item:
SECOND_VAR
index:
1018
Item:
Queue resource
index:
1019
Item:
/sys/fs/cgroup/cpu/cpu.cfs_quota_us
index:
1020
Item:
top command
index:
1021
Item:
Alice
index:
1022
Item:
Kubernetes Service Catalog
index:
1023
Item:
PersistentVolumeClaim 1
index:
1024
Item:
svc.cluster.local
index:
1025
Item:
DownwardAPI volume
index:
1026
Item:
Service specification
index:
1027
Item:
memory consumption
index:
1028
Item:
Kubernetes master
index:
1029
Item:
Mountable secrets

Item:
Bearer token
index:
1233
Item:
DNS resolver
index:
1234
Item:
podAffinityTerm
index:
1235
Item:
/api/v1/namespaces/foo/pods
index:
1236
Item:
/etc/downward/
index:
1237
Item:
kubectl apply command
index:
1238
Item:
Perl
index:
1239
Item:
binary values
index:
1240
Item:
os.hostname
index:
1241
Item:
hwrng
index:
1242
Item:
Role resource
index:
1243
Item:
sa
index:
1244
Item:
Running pod
index:
1245
Item:
scaleIO
index:
1246
Item:
Kubernetes Services
index:
1247
Item:
--data-checksums
index:
1248
Item:
PodSecurityPolicy Admission Control plugin
index:
1249
Item:
ServiceAccount usernames
index:
1250
Item:
latencyMillis
index:
1251
Item:
--cacert option
index:
1252
Item:
dynamic volume provisioner
index:
1253
Item:
selector.matchLabels
index:
1254
Item:
HTTP GET liveness probe
index:
1255
Item:
top
index:
1256
Item:
Kubernetes add-ons
index:
1257
Item:
VM (Virtual Machine)
index:
1258
Item:
Pod B2
index:
1259
Item:
SSH protocol
index:
1260
Item:
Pod specification
index:
1261
Item:
su

Item:
MySQL
index:
1466
Item:
ssl_certificate_key
index:
1467
Item:
watch mechanism
index:
1468
Item:
Target
index:
1469
Item:
Service Brokers
index:
1470
Item:
YAML definition
index:
1471
Item:
ifconfig eth0 down
index:
1472
Item:
fs.createWriteStream()
index:
1473
Item:
API aggregation
index:
1474
Item:
example.org/my-resource
index:
1475
Item:
client requests
index:
1476
Item:
/ui/
index:
1477
Item:
pd-standard
index:
1478
Item:
rollout history
index:
1479
Item:
kubia-0 pod
index:
1480
Item:
gpu
index:
1481
Item:
kubectl get pods command
index:
1482
Item:
cgroups system
index:
1483
Item:
some=thing
index:
1484
Item:
of=/dev/null
index:
1485
Item:
tty48
index:
1486
Item:
Application developer
index:
1487
Item:
--clusterrolebinding
index:
1488
Item:
Insufficient cpu
index:
1489
Item:
COMMAND
index:
1490
Item:
imaginary-kubia-website.yaml
index:
1491
Item:
externalTrafficPolicy
index:
1492
Item:
health check
index:
1493
Item:
minikubeVM
index:
1494
Item:
app-config
index:
1495
Item:
sc

Item:
NoSchedule effect
index:
1706
Item:
pod creation
index:
1707
Item:
CHANGE-CAUSE column
index:
1708
Item:
Minikube VM
index:
1709
Item:
ClusterAdmin
index:
1710
Item:
Log rotator
index:
1711
Item:
MaxUnavailable
index:
1712
Item:
Network between pods
index:
1713
Item:
kube-proxies
index:
1714
Item:
terminating
index:
1715
Item:
v1 API version
index:
1716
Item:
requestCount
index:
1717
Item:
extensions.example.com/v1
index:
1718
Item:
CONFIG_BAR
index:
1719
Item:
/apis/batch/v1
index:
1720
Item:
Backend container
index:
1721
Item:
Label Selectors
index:
1722
Item:
application logging
index:
1723
Item:
master node
index:
1724
Item:
external-ip
index:
1725
Item:
HTTP connection
index:
1726
Item:
dig DNS lookup tool
index:
1727
Item:
creation_method=manual label
index:
1728
Item:
env in (prod,devel)
index:
1729
Item:
SSDs
index:
1730
Item:
password file
index:
1731
Item:
kube-dns-v20-101vq
index:
1732
Item:
dialout
index:
1733
Item:
secretKeyRef
index:
1734
Item:
Scala
index:
1735
Ite

In [20]:
query_collection(collection_entity,"data plane and control plane",3)

{'ids': [['446', '1405', '907']],
 'distances': [[100.9042739868164, 167.74832153320312, 171.5594940185547]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['Control Plane components',
   'Kubernetes Control Plane components',
   'Control Plane pods']],
 'uris': None,
 'data': None}

In [22]:
collection_desc = client.delete_collection(name="entities_db_desc")
collection_desc = client.create_collection(name="entities_db_desc")

In [23]:
docs_lst=list(set(list(single_df_final['description'])))

In [24]:
collection_desc_data=chroma_collection_maker(collection_desc,docs_lst)

Item:
process of running batch jobs periodically or once in the future
index:
0
Item:
A feature that manages persistent volume labels
index:
1
Item:
A type of persistent storage for containers
index:
2
Item:
a type of Kubernetes resource that represents the kubia website
index:
3
Item:
the container running the backend process in a pod
index:
4
Item:
The process of increasing the resources (CPU and memory) available to a pod.
index:
5
Item:
a label key for specifying an availability zone
index:
6
Item:
Kubernetes scheduling function to spread pods across nodes
index:
7
Item:
A group of nodes that can be used for scheduling pods.
index:
8
Item:
a type of container that provides proxy functionality
index:
9
Item:
SSL certificate file
index:
10
Item:
Scope for the custom resource (namespaced)
index:
11
Item:
The specification of a Deployment, which includes properties like desired replica count and deployment strategy
index:
12
Item:
An operator that matches if the label's value is not an

Item:
Google Cloud Platform service
index:
116
Item:
an example network packet being routed through the service
index:
117
Item:
An object representing an HTTP request
index:
118
Item:
the container running the frontend process in a pod
index:
119
Item:
rule in PodSecurityPolicy that constrains user or group IDs
index:
120
Item:
An error that occurs within the application.
index:
121
Item:
Servers that resolve domain names to IP addresses.
index:
122
Item:
a containerization platform
index:
123
Item:
can be forwarded to the pod's ports 8080 and 8443 using a single multi-port service
index:
124
Item:
The version number of an API resource.
index:
125
Item:
allowing pods to bind to specific host ports
index:
126
Item:
A component that interacts with the Database service to provision and bind ServiceInstances
index:
127
Item:
Updated PVC definition with dynamic provisioning
index:
128
Item:
Command for retrieving service instance status
index:
129
Item:
Capability to ensure continuous oper

Item:
system log file
index:
228
Item:
programming language and runtime environment
index:
229
Item:
A section in a YAML manifest that specifies the metadata for a resource
index:
230
Item:
command to get logs of a specific container in a multi-container pod
index:
231
Item:
a specification for the pod affinity term
index:
232
Item:
Primary API server for Kubernetes
index:
233
Item:
Type of IP address assigned to a node
index:
234
Item:
A container running a web server
index:
235
Item:
A type of resource in Kubernetes that exposes services available in an application.
index:
236
Item:
Command used to create a Job resource
index:
237
Item:
The desired number of pods that should be running at any given time.
index:
238
Item:
Build tool used for Java projects
index:
239
Item:
Functions used to check node eligibility for pod scheduling
index:
240
Item:
API group for batch jobs
index:
241
Item:
The name of the container being created.
index:
242
Item:
A command used with gcloud to retrieve 

Item:
command to retrieve a Secret resource from a Kubernetes cluster
index:
343
Item:
components that enable additional features in a Kubernetes cluster
index:
344
Item:
token used for authorization with the API server
index:
345
Item:
Instruction in a Dockerfile for creating an empty image
index:
346
Item:
A Kubernetes resource that specifies limits for pods and containers.
index:
347
Item:
Label selector for pods with env label not set to prod or devel
index:
348
Item:
Allow pods to run on tainted nodes
index:
349
Item:
30
index:
350
Item:
Used for requests where none of the authentication plugins could authenticate the client.
index:
351
Item:
Name of the broker providing a service
index:
352
Item:
The request sent by the client to the API server.
index:
353
Item:
Output stream where the application logs are written
index:
354
Item:
enable a StatefulSet to stamp out PersistentVolumeClaims along with each pod instance
index:
355
Item:
Components that modify iptables rules
index:
356

Item:
Resource that provides a network identity and load balancing for accessing applications
index:
451
Item:
running a bash shell in an existing pod's container
index:
452
Item:
a Kubernetes node running on machine B
index:
453
Item:
A Clojure client library for Kubernetes
index:
454
Item:
A Docker image containing the DNS utilities.
index:
455
Item:
destination IP address of packet sent by Pod A
index:
456
Item:
a file containing configuration data in JSON format
index:
457
Item:
Request for a service to be provisioned by the team
index:
458
Item:
an option for curl to follow redirects
index:
459
Item:
method for sending a response back to the client, used in Node.js applications
index:
460
Item:
the process of writing logs from an application
index:
461
Item:
data duplication command used to generate load on the system
index:
462
Item:
a toleration effect that prevents pods from running on a node if it is unreachable or not ready
index:
463
Item:
A Ruby client library for Kubernete

...

Item:
A list of containers in the pod template
index:
1226
Item:
The field in the ReplicationController's definition that specifies the label selector for the pods managed by the controller.
index:
1227
Item:
Path to secret volume holding certificate and key files
index:
1228
Item:
Abbreviation for ReplicationController.
index:
1229
Item:
Definition of a container in the pod template
index:
1230
Item:
resources that represent running containers in the cluster
index:
1231
Item:
A deployment strategy where a new version of an application is deployed for testing and feedback
index:
1232
Item:
A secret that provides authentication tokens for the StatefulSet.
index:
1233
Item:
A command used to create a HorizontalPodAutoscaler (HPA) object and enable horizontal autoscaling.
index:
1234
Item:
A command to create a Google Compute Engine persistent disk.
index:
1235
Item:
Java method used to retrieve an environment variable
index:
1236
Item:
PostgreSQL database port number
index:
1237
Item:
A 

Item:
A list of subsets for the Endpoints resource.
index:
1328
Item:
output of kubectl delete command indicating successful deletion
index:
1329
Item:
A component that provisions PersistentVolumes dynamically
index:
1330
Item:
web server software used in the example
index:
1331
Item:
the directory within the volume where the repository will be cloned
index:
1332
Item:
A digital certificate used to authenticate the API server.
index:
1333
Item:
an API endpoint for accessing ConfigMap data
index:
1334
Item:
Persistent Volume Claim for replica set A2
index:
1335
Item:
address of the Kubernetes server
index:
1336
Item:
A target value for QPS
index:
1337
Item:
default
index:
1338
Item:
the name of a pod in the example
index:
1339
Item:
a role binding name
index:
1340
Item:
components that handle requests to custom API objects
index:
1341
Item:
Placeholder for first node's IP address
index:
1342
Item:
The client's IP address that is not preserved when connecting to a service through a node 

Item:
system monitoring command that displays memory and CPU usage statistics
index:
1441
Item:
An HTTP method used to update or replace a resource.
index:
1442
Item:
newline character used to denote multi-line values
index:
1443
Item:
command to see pod's log (more precisely, container's log)
index:
1444
Item:
An API endpoint that lists pods across all namespaces.
index:
1445
Item:
A hook executed immediately after a container starts.
index:
1446
Item:
a command to list running pods with their IPs
index:
1447
Item:
a NetworkPolicy resource for allowing access to the shopping cart microservice
index:
1448
Item:
directory containing configuration files used to create a ConfigMap
index:
1449
Item:
Resource limits for pods, which can be limited by a ResourceQuota.
index:
1450
Item:
A YAML or JSON file that defines a pod's configuration, including its metadata and status.
index:
1451
Item:
A method to add labels to a pod
index:
1452
Item:
A feature in Kubernetes that automatically scales t

Item:
High-level object representing a messaging broker
index:
1551
Item:
file containing pod name
index:
1552
Item:
The version of the Kubernetes API used for batch processing
index:
1553
Item:
An authentication token sent in the Authorization header to authenticate with the API server.
index:
1554
Item:
API server feature used by the Scheduler and Kubelet
index:
1555
Item:
A client outside the Kubernetes cluster that can access services exposed to it through NodePort, LoadBalancer, or Ingress resources.
index:
1556
Item:
a pod with a ConfigMap volume mounted to it
index:
1557
Item:
A condition that is displayed when the rollout can't make progress within the specified deadline.
index:
1558
Item:
Plugins that validate and/or modify resources in requests.
index:
1559
Item:
Mark nodes with specific conditions
index:
1560
Item:
command used to execute a command inside a running container with interactive shell
index:
1561
Item:
A validation schema to validate the contents of the YAML.
in

Item:
PersistentVolumes names
index:
1653
Item:
Port number and protocol for the Service resource
index:
1654
Item:
Information about the pod being created, including its name.
index:
1655
Item:
Flag indicating whether a plan can be updated
index:
1656
Item:
One or more pod replicas created and started according to the Job's pod template.
index:
1657
Item:
A specification for defining APIs in a machine-readable format
index:
1658
Item:
property in Kubernetes Pod configuration to drop specific capabilities
index:
1659
Item:
the policy for restarting a container when it crashes
index:
1660
Item:
a list of values in the matchExpressions that specifies the value to match for the node selector
index:
1661
Item:
Node port number for the Service resource
index:
1662
Item:
A YAML file used to define a pod that binds to a port on the node.
index:
1663
Item:
The name of the Secret that will store the credentials for accessing the service
index:
1664
Item:
A field in the CronJob specification tha

In [25]:
query_collection(collection_desc_data,"data plane and control plane",3)

{'ids': [['1085', '968', '515']],
 'distances': [[150.1617431640625, 195.83901977539062, 223.19151306152344]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['Node that runs the Control Plane components',
   'a way to elect a leader in control plane components',
   'Components in the Kubernetes control plane that monitor and respond to changes in cluster resources.']],
 'uris': None,
 'data': None}

In [26]:
collection_cat = client.delete_collection(name="entities_db_cat")
collection_cat = client.create_collection(name="entities_db_cat")

In [27]:
docs_lst=list(set(list(single_df_final['category'])))

In [28]:
collection_cat_data=chroma_collection_maker(collection_cat,docs_lst)

Item:
metadata
index:
0
Item:
status
index:
1
Item:
output
index:
2
Item:
process,container
index:
3
Item:
filesystem
index:
4
Item:
selector
index:
5
Item:
binary
index:
6
Item:
process/thread
index:
7
Item:
message
index:
8
Item:
error
index:
9
Item:
Filesystem
index:
10
Item:
Shell
index:
11
Item:
specification
index:
12
Item:
programming language
index:
13
Item:
book
index:
14
Item:
parameter
index:
15
Item:
pod
index:
16
Item:
Network
index:
17
Item:
personnel
index:
18
Item:
system
index:
19
Item:
role
index:
20
Item:
controller
index:
21
Item:
tool
index:
22
Item:
certificate
index:
23
Item:
Software/Command
index:
24
Item:
arguments
index:
25
Item:
data format
index:
26
Item:
Pod Status
index:
27
Item:
Hardware/Location
index:
28
Item:
template
index:
29
Item:
software development
index:
30
Item:
docker
index:
31
Item:
algorithm
index:
32
Item:
http-method
index:
33
Item:
capability
index:
34
Item:
concept
index:
35
Item:
string
index:
36
Item:
attribute
index:
37
Item:
method


In [29]:
query_collection(collection_cat_data,"Replication Controller",3)

{'ids': [['21', '167', '80']],
 'distances': [[218.0276336669922, 254.6217458146975, 258.33135986328125]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['controller', 'software/protocol', 'network,process']],
 'uris': None,
 'data': None}

In [32]:
graph_output_dir="./graph_output_networkx/"
input_graph="graph_phase1_with_pages_text.graphml"
import networkx as nx

G=nx.read_graphml(graph_output_dir+input_graph)

In [34]:
def single_occur_node_builder(entity,description,category):
    #print(entity)
    if (entity!="") or (entity!='') or (entity!='\n') or (entity!=" ") or (entity!=nan) or (entity!='/n'):
        print(entity)
        G.add_node(entity,  title=entity , description=description, category=category,color="cyan") 
               

In [35]:
single_df[['entity','description','category']].apply(lambda x: single_occur_node_builder(x.entity,
                                                                                                 x.description,
                                                                                                 x.category
                                                                                                              ),axis=1)

UTS namespace
Port space
Flat inter-pod network
NAT (Network Address Translation)
Worker nodes
LAN (Local Area Network)
Software-defined network
VM (Virtual Machine)
App
Container 1
Container 2
Log rotator
Collector
Data processor
Communication adapter
Frontend process
Backend process
Frontend container
Backend container
REST API
API object definitions
Kubernetes API reference documentation
Pod metadata
Pod specification
dnsPolicy
nodeName
serviceAccount
terminationGracePeriodSeconds
containerID
lastState
ready
restartCount
state
hostIP
phase
podIP
startTime
Kubernetes API version
Spec
Pod specification/contents
Detailed status of the pod and its containers
protocol
status
po kubia-manual
-o yaml
-o json
process's standard output
standard error stream
$ docker logs <container id>
kubectl logs kubia-manual
Kubia server starting...
-c <container name>
$ kubectl logs kubia-manual -c kubia
container name
port forwarding
localhost:8888
8080
Resource
Microservice
UI Pod
canary release
stable

0       None
1       None
2       None
3       None
4       None
        ... 
1843    None
1844    None
1845    None
1846    None
1847    None
Length: 1826, dtype: object

In [36]:
G.nodes

NodeView(('Kubernetes_in_Action.pdf', '89', '1', 'text_0', '90', 'text_1', '91', 'text_2', '92', 'text_3', '93', 'text_4', '94', 'text_5', '95', 'text_6', '96', 'text_7', '97', 'text_8', '98', 'text_9', '99', 'text_10', '100', 'text_11', '101', 'text_12', '102', 'text_13', '103', 'text_14', '104', 'text_15', '105', 'text_16', '106', 'text_17', '107', 'text_18', '108', 'text_19', '109', 'text_20', '110', 'text_21', '111', 'text_22', '112', 'text_23', '113', 'text_24', '114', 'text_25', '115', 'text_26', '116', 'text_27', '117', 'text_28', '118', 'text_29', '119', 'text_30', '120', 'text_31', '121', 'text_32', '122', 'text_33', '123', 'text_34', '124', 'text_35', '125', 'text_36', '126', 'text_37', '127', 'text_38', '128', 'text_39', '129', 'text_40', '130', 'text_41', '131', 'text_42', '132', 'text_43', '133', 'text_44', '134', 'text_45', '135', 'text_46', '136', 'text_47', '137', 'text_48', '138', 'text_49', '139', 'text_50', '140', 'text_51', '141', 'text_52', '142', 'text_53', '143',

In [37]:
import pickle
#####Linking Single Occurence entities to pages ##########
def link_entities_with_page(G,document_dict):
    
    # Load data (deserialized data) into serilaizable list of of data#####
###This acts as the source for the Graph Building ######

    with open(document_dict, 'rb') as handle:
        document_dict_deserialized = pickle.load(handle)
    
    doc_len=len(document_dict_deserialized)
    
    for page_idx in range(0,doc_len):
        print(page_idx)
        #Adding the Page Number Nodes and connecting then to the root Node that is the document_name name node
        #Form the page number from the dictionary metadata 'page'
        entity_list=document_dict_deserialized[page_idx]['entities']
        print(entity_list)
        page_num="Page_"+ str(document_dict_deserialized[page_idx]['page'])
        #G.add_node(page_idx, label=page_num , title=page_num,color="blue")
        #G.add_edge(1,page_idx)
    
        ##Under each Page we will add the different types of extracted entities like tables,text,images
        #We will Create and add the text node and attach it with the page_num node.
        #text_idx='text_'+str(i)
        #G.add_node(text_idx, label=document_dict_deserialized[i]['text'] , title=document_dict_deserialized[i]['text'],color="violet")
        #G.add_edge(page_idx,text_idx)
        print(page_num)
        print(document_dict_deserialized[page_idx]['entities'])
    
    
        for entity in entity_list:
            if entity['entity'] in G.nodes:
                print('entity:')
                print(entity['entity'])
                G.add_edge(page_idx,entity['entity'],name='contains')
            else:
                print("The Entity is a kind of entity which has occured multiple times ,hence will be handled later ")
    #entities_list_doc.extend(document_dict_deserialized[i]['entities'])
    return G

In [38]:
single_df.to_csv(graph_output_dir+'Single_Entities_embeddings.csv')

In [39]:
document_path="../pdf_enrichment/pdf_enriched_output/"
document_dict_deserialized='pdf_enriched_content_dict_phase5_extract_highligts_478_final.pickle'
document_full_path=document_path+document_dict_deserialized
G=link_entities_with_page(G,document_full_path)

0
[{'entity': 'Docker', 'description': 'Containerization platform', 'category': 'software'}, {'entity': 'Kubernetes', 'description': 'Container orchestration system', 'category': 'software'}, {'entity': 'Pods', 'description': 'Group of containers that share resources and run as a single unit', 'category': 'application'}, {'entity': 'Containers', 'description': 'Isolated execution environment for processes', 'category': 'container'}, {'entity': 'Linux namespaces', 'description': 'Resource isolation mechanism in Linux', 'category': 'process'}, {'entity': 'Network namespace', 'description': 'Shared network resources among containers in a pod', 'category': 'network'}, {'entity': 'UTS namespace', 'description': 'Shared hostname and network interfaces among containers in a pod', 'category': 'network'}, {'entity': 'IPC namespace', 'description': 'Shared inter-process communication resources among containers in a pod', 'category': 'process'}, {'entity': 'PID namespace', 'description': 'Shared 

In [44]:
import networkx as nx
nx.write_graphml_lxml(G, graph_output_dir+"graph_phase2_with_single_entity_desc_embeddings.graphml")

In [45]:
nx.write_gml(G, graph_output_dir+"graph_phase2_with_single_entity_desc_embeddings.gml")

In [42]:
from pyvis.network import Network
####The main contents of Kubernets In Action book is contained from page 87 to 565####
####Thats why we will only take the sliced portion i.e. from page 87 to 565 of the pdf dictionary list#####
nt = Network('800px', '2000px',notebook=True,cdn_resources='in_line')
# populates the nodes and edges data structures
nt.from_nx(G)

In [43]:
nt.toggle_physics(True)
nt.show(graph_output_dir+'graph_stage2_with_single_entity_desc_embeddings.html')
#nt.show_buttons(filter_=['physics'])
#nt.save('nx2.html')
#nt.write_html('graph_stage1.html')

./graph_output_networkx/Graph_Stage2_with_single_entity_desc_embeddings.html
